In [ ]:
####################################################################################
# Spotify Account: ZoLTaR                                                          #
####################################################################################
# User ID: 125165874                                                               #
# OAuth token: BQAdyJl8AI8njMGX0aJmqdFd3YDUTUNtw                                   #
#              _KFHAUOcO8LdVFojS_PgdnvcrvRwSYvCj                                   #
#              eiSAGwm9mwOGZSVjzUxICgPOIoEffQ31s                                   #
#              X2l-IGvDFULG7ZmAdmZhhruAgi7o_4W_n                                   #
#              HpFjfde7N-H7YjFai-i0PSA9j2OCvLQjq                                   #
#              bNkeHjw-Q                                                           #
# App - ZoLTiFy                                                                    #
# ID / Secret: 085733e14f894738bf40334443020126 / 62d8f8eccfdf49d9bf8710eb47d2a34c #
# App - Releases_2022                                                              #
# ID / Secret: bf7349672eab4050bf4567ed7c6732ec / bda8d2acbf974cd5932f00a6a763fad3 #
# App - 3                                                                          #
# ID / Secret: dbf80683949a456592837fe493b579a7 / ac2a80f3cde94feb8c4b886cfd556cfd #
# App - 4                                                                          #
# ID / Secret: b6f601f082f7491aa9c4ab7271fae6fd / fc0e21ebc5974886acc6ba14d4676c02 #
# App - 5                                                                          #
# ID / Secret: 0464cab38a1a4fb89c8678f6b1e27e81 / 58c42925f46941c6b7ca21cb157908b6 #
# App - 6                                                                          #
# ID / Secret: 65df35b84b8a47d2b66e21a2a8e39dd3 / 90e3ed69a2f24735a5d4710bc6a377fa #
# App - 7                                                                          #
# ID / Secret: f61219e4380f4b8796e9f4d224da5494 / cb4cb64077834ba2bae059bf983dd47f #
# App - 8                                                                          #
# ID / Secret: 3c51ca3a9e314bd5aeb608d22e369efd / 174b4c06e0974b01bbf7294d11c9bd7f #
# App - 9                                                                          #
# ID / Secret: 3f3a6839a5de407ab16c87ba3ba8cd68 / 03410f4b5165403b8219d14cc9550605 #
# App - 10                                                                         #
# ID / Secret: a8461dc61ff44abd83f94a4a07de530f / ed2e1d6898834e4bb649df080e5474bf #
#                                                                                  #
####################################################################################


############################
# Spotify account: Unknown #
############################

# App - Unknown
# Client ID: 7a06f1d458ac41f8b25577e02ec30d62
# Client Secret: 2f10949260a449968cb0fa13f1e5a4c9

### Import des bibliothèques requises + Initialisation du Client Spotipy 

In [ ]:
import spotipy, threading, itertools, time, datetime
import pandas as pd

from collections import defaultdict
from spotipy.oauth2 import SpotifyClientCredentials

client_id = ['085733e14f894738bf40334443020126', 'bf7349672eab4050bf4567ed7c6732ec', 'dbf80683949a456592837fe493b579a7', 'b6f601f082f7491aa9c4ab7271fae6fd', '0464cab38a1a4fb89c8678f6b1e27e81', '65df35b84b8a47d2b66e21a2a8e39dd3', 'f61219e4380f4b8796e9f4d224da5494', '3c51ca3a9e314bd5aeb608d22e369efd', '3f3a6839a5de407ab16c87ba3ba8cd68', 'a8461dc61ff44abd83f94a4a07de530f', '7a06f1d458ac41f8b25577e02ec30d62']
client_secret = ['62d8f8eccfdf49d9bf8710eb47d2a34c', 'bda8d2acbf974cd5932f00a6a763fad3', 'ac2a80f3cde94feb8c4b886cfd556cfd', 'fc0e21ebc5974886acc6ba14d4676c02', '58c42925f46941c6b7ca21cb157908b6', '90e3ed69a2f24735a5d4710bc6a377fa', 'cb4cb64077834ba2bae059bf983dd47f', '174b4c06e0974b01bbf7294d11c9bd7f', '03410f4b5165403b8219d14cc9550605', 'ed2e1d6898834e4bb649df080e5474bf', '2f10949260a449968cb0fa13f1e5a4c9']

spotify_client_credentials = []
spotify_client = []

for i in range(len(client_id)):
      spotify_client_credentials.append(SpotifyClientCredentials(client_id=client_id[i], client_secret=client_secret[i]))

for cred in spotify_client_credentials:                                                    
      spotify_client.append(spotipy.Spotify(client_credentials_manager = spotify_client_credentials[0]))

### Chargement des données préalablement extraites (source : MusicBrainz)

In [ ]:
input_data = pd.read_csv('mb_data/albums_released_2022.csv', sep='\t')
input_data

### Retrieve Data from Spotify API

In [ ]:
# Initialization of main variables
track_db_without_af = []
af_db = []

# Input dataframe - Select a slice for testing => ie. dataframe[:100]
input_data = input_data
input_main_data_type = 'artist'
input_secondary_data_type = 'track'

# Multithreading - Data slicing to feed the threads
number_of_threads = len(spotify_client)
slice_size = len(input_data) // (number_of_threads)
slice_limit = []
thread = []

SLEEPTIME = 3
NOMAX = 1e4
MAX = 5

for i in range(number_of_threads + 1):
    if i == 0:
        slice_limit.append(0)
    elif i != number_of_threads:
        slice_limit.append(i*slice_size)
    else:
        slice_limit.append(len(input_data))

def get_audio_features(track_ids, thread_number):
    audio_features = []

    if len(track_ids) > 100:
        print("TOO MANY TRACK IDS")
        return audio_features
    else:
        audio_features = spotify_client[thread_number].audio_features(track_ids)
        audio_features = [item for item in audio_features if item is not None]

    for idx, item in enumerate(audio_features):
        item['track_id'] = item.pop('uri')
        item.update({'track_id': item['track_id'].split(':')[-1]})

    return audio_features
        
def launch_thread(start, end, thread_number):
    # try:
        request = 0
        max_consecutive_requests = NOMAX

        for index in range(start, end):
            request += 1
            row = input_data.iloc[index]
            
            if request % max_consecutive_requests == 1:
                time.sleep(SLEEPTIME)
            
            print(input_secondary_data_type + ": " + row[input_secondary_data_type], input_main_data_type + ": " + row[input_main_data_type])
            print("Thread n°", thread_number + 1, " -- Request n°", request)

            track_results = spotify_client[thread_number].search(q=input_secondary_data_type + ":" + row[input_secondary_data_type] + ' ' + input_main_data_type + ': ' + row[input_main_data_type], type='track', limit=50)

            if track_results['tracks']['total'] == 0:
                continue

            track_ids_for_batch_AF_request = []

            for i, t in enumerate(track_results['tracks']['items']):

                track_info = dict(track_id = t['id'], track_name = t['name'], artist_id = t['artists'][0]['id'],
                                  artist_name = t['artists'][0]['name'], album_id = t['album']['id'],
                                  album_name = t['album']['name'], release_date = t['album']['release_date'],
                                  popularity = t['popularity'], duration_ms = t['duration_ms'], explicit = t['explicit'],
                                  external_urls = t['external_urls']['spotify'], is_local = t['is_local'],
                                  preview_url = t['preview_url'], track_number = t['track_number'], genres = '',
                                  danceability = None, energy = None, key = None, loudness = None, mode = None,
                                  speechiness = None, acousticness = None, instrumentalness = None, liveness = None,
                                  valence = None, tempo = None, time_signature = None)
                
                if len(track_info['release_date']) == 10:
                    if datetime.datetime.strptime(track_info['release_date'], '%Y-%m-%d').date().year == 2022:
                        track_db_without_af.append(track_info)
                        track_ids_for_batch_AF_request.append(t['id'])
                elif len(track_info['release_date']) == 4:
                    if int(track_info['release_date']) == 2022:
                        track_db_without_af.append(track_info)
                        track_ids_for_batch_AF_request.append(t['id'])
                else:
                    print("CKOICETRUC ? =>", track_info['release_date'])
                
            if len(track_ids_for_batch_AF_request) > 100:
                raise Exception("Expected - Development Exception: more than 100 audio features to recover (feature in development)")
            else:
                af_db.extend(get_audio_features(track_ids_for_batch_AF_request, thread_number))

    # except Exception as e:
    #     print("Exception raised for thread N°", thread_number,":", e)

# Launch the multithreading
for i in range(number_of_threads):
    thread.append(threading.Thread(target=launch_thread, args=(slice_limit[i], slice_limit[i+1], i)))
    thread[i].start()

# Wait for completion
for i in range(number_of_threads):
    thread[i].join()

In [ ]:
# def merge(shared_key, *iterables):
#     result = defaultdict(dict)
#     for dictionary in itertools.chain.from_iterable(iterables):
#         result[dictionary[shared_key]].update(dictionary)
#     for dictionary in result.values():
#         dictionary.pop(shared_key)
#     return result

track_df_with_af = pd.DataFrame()
af_df = pd.DataFrame(af_db)
track_df_without_af = pd.DataFrame(track_db_without_af)
print(track_df_without_af.info(), '\n', track_df_without_af)
track_df_without_af.drop_duplicates(ignore_index=True, subset=['track_name', 'duration_ms'], keep = 'first', inplace = True)

print(len(af_df)," = ",len(track_df_without_af)," = ",len(track_df_with_af))
track_df_with_af = pd.merge(track_df_without_af, af_df, on='track_id', how='inner')
print(len(af_df)," = ",len(track_df_without_af)," = ",len(track_df_with_af))

In [ ]:
track_df_with_af.to_csv('extraction_test.csv', index=False)